# Make Your Own Database

The COSIMA cookbook uses a database to access information about experiments and to help with loading model output. We maintain a default database for ACCESS-OM2 experiments, but there are occasions when you might want to make your own database. This tutorial outlines the process of making your own private database.

**Requirements:** We recommend that you use the most recent `conda/analysis3` (or later) kernel on NCI (or your own up-to-date cookbook installation).

In [ ]:
%matplotlib inline
import cosima_cookbook as cc

**First, create a database session** using the inbuilt `create_session` function. To do this, you need to specify a path for the database - choose a location where you have write permission (that is, not the example that I have given here):

In [ ]:
db = 'local_cc_test.db'
session = cc.database.create_session(db)

Note that you need to create the database session every time you start up your notebook; you can then update this database however many times you like.

**Now you are ready to build a database.** First, select which *experiments* you want to include in your database. For these purposes, an *experiment* is a directory containing output from a single simulation. (If you use a higher level directory you won't be able to distinguish between experiments.) 

My example below constructs a list of three experiment directories; I have chosen three cases with differing resolution. The database will be built to index all netcdf files in each directory.

In [ ]:
dir_list=['/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_B1',
          '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_iaf_gmredi6',
          '/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_iaf']
cc.database.build_index(dir_list,session,update=True)

Note that this operation may take a little while first time through, but is relatively painless to update -- **provided that you have the `update=True` flag switched on**. Now you have your own database - remember to specify your own database when you load model output, or else it will look for your experiment in the default database.

## Using the database
To know how to effectively use this database, please see the companion tutorial: `COSIMA_CookBook_Tutorial`. Alternatively, here is a sample that shows how you might load a variable from an experiment in your database.

In [ ]:
expt =  '025deg_jra55v13_iaf_gmredi6'
variable = 'ke_tot'
darray = cc.querying.getvar(expt, variable, session)
annual_average = darray.resample(time='A').mean(dim='time')
annual_average.plot()

If you want to know more about the inbuilt functions used above, you can use the help function at any time, for example:

In [ ]:
help(cc.database.create_session)

In [ ]:
help(cc.querying.getvar)